bce loss

In [19]:
def baselines():
    from ProgressPrinter import ProgressPrinter
    from FewShot import ZeroShotClassifier, FewShotClassifier, PEFTFewShotClassifier
    from PersonalizedCitation import train_loader
    from peft import IA3Config, TaskType
    import torch

    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    zero = ZeroShotClassifier()
    few = FewShotClassifier()
    peft_config = IA3Config(task_type=TaskType.CAUSAL_LM, fan_in_fan_out=True)
    ft = PEFTFewShotClassifier(peft_config)

    with ProgressPrinter('0 shot acc', '1 shot acc', 'ft acc', 'ft loss') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                multichoices = [ ( i, train.choices ) for i in inputs ] 
                zeroguesses = zero(multichoices)
                zeroreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(zeroguesses, answers) ]).mean().item()

                shots = []
                for input_embedding, profile in zip(train.embed(inputs), profiles):
                    profile_embeddings = train.embeddings(profile)
                    indices = torch.topk(input_embedding @ profile_embeddings.T, dim=0, k=1).indices.to('cpu')
                    shots.append(train.stringify_articles(indices))

                fewguesses = few(multichoices, shots)[0]
                fewreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(fewguesses, answers) ]).mean().item()

                ftguesses, ftguessindices = ft(multichoices, shots)
                ftreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(ftguesses, answers) ])

            loss = ft.bandit_learn(multichoices, shots, ftguessindices, ftreward.to(device))
                
            printer.addobs(zeroreward, fewreward, ftreward.mean().item(), loss)

from Fork import SubProcess
with SubProcess() as process: process.parent or baselines()

n          0 shot acc      since 1 shot acc      since     ft acc      since    ft loss      since     dt (s)
1                   1          1          1          1          1          1      0.159      0.159        1.9
2                 0.5          0        0.5          0        0.5          0       2.92       5.68       2.91
4               0.625       0.75      0.625       0.75      0.625       0.75       1.78      0.632       5.69
8               0.688       0.75      0.688       0.75      0.625      0.625        1.2      0.622       9.35
16              0.594        0.5      0.594        0.5        0.5      0.375       1.06      0.927       16.3
32                0.5      0.406        0.5      0.406      0.531      0.562      0.893      0.722       31.4
64              0.453      0.406      0.453      0.406      0.547      0.562      0.805      0.717       64.7
128             0.488      0.523      0.488      0.523      0.516      0.484      0.761      0.717        126
256       

In [ ]:
def baselines():
    from ProgressPrinter import ProgressPrinter
    from FewShot import ZeroShotClassifier, FewShotClassifier, PEFTFewShotClassifier
    from PersonalizedCitation import train_loader
    from ReplayWrapper import ReplayWrapper
    from peft import IA3Config, TaskType
    import torch

    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    zero = ZeroShotClassifier()
    few = FewShotClassifier()
    peft_config = IA3Config(task_type=TaskType.CAUSAL_LM, fan_in_fan_out=True)
    ft = ReplayWrapper(PEFTFewShotClassifier(peft_config), replay_count=2, nbatches=64)

    with ProgressPrinter('0 shot acc', '1 shot acc', 'ft acc', 'ft loss') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                multichoices = [ ( i, train.choices ) for i in inputs ] 
                zeroguesses = zero(multichoices)
                zeroreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(zeroguesses, answers) ]).mean().item()

                shots = []
                for input_embedding, profile in zip(train.embed(inputs), profiles):
                    profile_embeddings = train.embeddings(profile)
                    indices = torch.topk(input_embedding @ profile_embeddings.T, dim=0, k=1).indices.to('cpu')
                    shots.append(train.stringify_articles(indices))

                fewguesses = few(multichoices, shots)[0]
                fewreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(fewguesses, answers) ]).mean().item()

                ftguesses, ftguessindices = ft.predict(multichoices, shots)
                ftreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(ftguesses, answers) ])

            loss = ft.bandit_learn((multichoices, shots), ftguessindices, ftreward.to(device))
                
            printer.addobs(zeroreward, fewreward, ftreward.mean().item(), loss)

from Fork import SubProcess
with SubProcess() as process: process.parent or baselines()

n          0 shot acc      since 1 shot acc      since     ft acc      since    ft loss      since     dt (s)
1                   1          1          1          1          1          1      0.159      0.159       2.09
2                 0.5          0        0.5          0        0.5          0       2.92       5.68       3.33
4               0.625       0.75      0.625       0.75      0.625       0.75       1.78      0.632       6.92
8               0.688       0.75      0.688       0.75      0.625      0.625        1.2      0.622       11.5
16              0.594        0.5      0.594        0.5        0.5      0.375       1.06      0.927       19.7
32                0.5      0.406        0.5      0.406      0.531      0.562      0.892      0.722       38.5
64              0.453      0.406      0.453      0.406      0.547      0.562      0.805      0.717       80.9
128             0.488      0.523      0.488      0.523      0.562      0.578      0.752      0.699        209
256       

policy gradient -- needs to be run with (2*r-1)

In [36]:
def baselines():
    from ProgressPrinter import ProgressPrinter
    from FewShot import ZeroShotClassifier, FewShotClassifier, PEFTFewShotClassifier
    from PersonalizedCitation import train_loader
    from ReplayWrapper import ReplayWrapper
    from peft import IA3Config, TaskType
    import torch

    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    zero = ZeroShotClassifier()
    few = FewShotClassifier()
    peft_config = IA3Config(task_type=TaskType.CAUSAL_LM, fan_in_fan_out=True)
    ft = ReplayWrapper(PEFTFewShotClassifier(peft_config), replay_count=2, nbatches=64)

    with ProgressPrinter('0 shot acc', '1 shot acc', 'ft acc', 'ft loss') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                multichoices = [ ( i, train.choices ) for i in inputs ] 
                zeroguesses = zero(multichoices)
                zeroreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(zeroguesses, answers) ]).mean().item()

                # shots = []
                # for input_embedding, profile in zip(train.embed(inputs), profiles):
                #     profile_embeddings = train.embeddings(profile)
                #     indices = torch.topk(input_embedding @ profile_embeddings.T, dim=0, k=1).indices.to('cpu')
                #     shots.append(train.stringify_articles(indices))
                shots = [""]*len(profiles)

                fewguesses = few(multichoices, shots)[0]
                fewreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(fewguesses, answers) ]).mean().item()

                ftguesses, ftguessindices = ft.predict(multichoices, shots)
                ftreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(ftguesses, answers) ])

            loss = ft.bandit_learn((multichoices, shots), ftguessindices, (2*ftreward-1).to(device))
                
            printer.addobs(zeroreward, fewreward, ftreward.mean().item(), loss)

from Fork import SubProcess
with SubProcess() as process: process.parent or baselines()

n          0 shot acc      since 1 shot acc      since     ft acc      since    ft loss      since     dt (s)
1                   1          1          1          1          1          1      0.214      0.214      0.803
2                 0.5          0        0.5          0        0.5          0      0.106   -0.00237       1.04
4               0.625       0.75      0.625       0.75      0.625       0.75     0.0539    0.00188       1.52
8               0.688       0.75      0.688       0.75      0.688       0.75     0.0275    0.00103       2.46
16              0.594        0.5      0.594        0.5      0.594        0.5     0.0138   3.87e-05       4.31
32                0.5      0.406        0.5      0.406        0.5      0.406    0.00675  -0.000255       8.15
64              0.453      0.406      0.453      0.406      0.453      0.406      0.003  -0.000753       15.6
128             0.488      0.523      0.488      0.523      0.488      0.523    0.00189   0.000773       43.9
256       

In [22]:
def baselines():
    from ProgressPrinter import ProgressPrinter
    from FewShot import ZeroShotClassifier, FewShotClassifier, PEFTFewShotClassifier
    from PersonalizedCitation import train_loader
    from peft import IA3Config, TaskType
    import torch

    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    zero = ZeroShotClassifier()
    few = FewShotClassifier()
    peft_config = IA3Config(task_type=TaskType.CAUSAL_LM, fan_in_fan_out=True)
    ft = PEFTFewShotClassifier(peft_config)

    with ProgressPrinter('0 shot acc', '1 shot acc', 'ft acc', 'ft loss') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                multichoices = [ ( i, train.choices ) for i in inputs ] 
                zeroguesses = zero(multichoices)
                zeroreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(zeroguesses, answers) ]).mean().item()

                shots = []
                for input_embedding, profile in zip(train.embed(inputs), profiles):
                    profile_embeddings = train.embeddings(profile)
                    indices = torch.topk(input_embedding @ profile_embeddings.T, dim=0, k=1).indices.to('cpu')
                    shots.append(train.stringify_articles(indices))

                fewguesses = few(multichoices, shots)[0]
                fewreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(fewguesses, answers) ]).mean().item()

                ftguesses, ftguessindices = ft(multichoices, shots)
                ftreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(ftguesses, answers) ])

            loss = ft.bandit_learn(multichoices, shots, ftguessindices, ftreward.to(device))
                
            printer.addobs(zeroreward, fewreward, ftreward.mean().item(), loss)

from Fork import SubProcess
with SubProcess() as process: process.parent or baselines()

n          0 shot acc      since 1 shot acc      since     ft acc      since    ft loss      since     dt (s)
1                   1          1          1          1          1          1     -0.319     -0.319       1.87
2                 0.5          0        0.5          0        0.5          0     -0.159          0       2.87
4               0.625       0.75      0.625       0.75      0.625       0.75    -0.0823   -0.00517       5.63
8               0.688       0.75      0.688       0.75      0.688       0.75    -0.0423   -0.00223       9.27
16              0.594        0.5      0.594        0.5      0.594        0.5    -0.0216   -0.00094         16
32                0.5      0.406        0.5      0.406        0.5      0.406     -0.011  -0.000395       30.9
64              0.453      0.406      0.453      0.406      0.453      0.406    -0.0056  -0.000206       63.7
128             0.488      0.523      0.488      0.523      0.488      0.523   -0.00285  -9.22e-05        124
256       